In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pandas as pd
import jieba
import re

In [2]:
class MyDataset(Dataset):
    def __init__(self, max_len, debug=True):
        super().__init__()
        df = pd.read_csv('../../datasets/THUCNews/train.csv')
        df = df.dropna()
        if debug:
            df = df.sample(2000).reset_index(drop=True)
        else:
            df = df.sample(50000).reset_index(drop=True)
        # 读取常用停用词
        stopwords = [line.strip() for line in open('../../stopwords/cn_stopwords.txt', 'r', encoding='utf-8').readlines()]
        sentences = []
        for title in df['title']:
            # 去除标点符号
            title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
            # jieba分词
            sentence_seged = jieba.cut(title.strip())
            outstr = ''
            for word in sentence_seged:
                if word != '\t' and word not in stopwords:
                    outstr += word
                    outstr += ' '
            if outstr != '':
                sentences.append(outstr)
        # 获取所有词(token), <pad>用来填充不满足max_len的句子
        token_list = ['<pad>'] + list(set(' '.join(sentences).split()))
        # token和index互转字典
        self.token2idx = {token: i for i, token in enumerate(token_list)}
        self.idx2token = {i: token for i, token in enumerate(token_list)}
        self.vocab_size = len(self.token2idx)

        self.inputs = []
        for sentence in sentences:
            tokens = sentence.split()
            input_ = [self.token2idx[token] for token in tokens]
            if len(input_) < max_len:
                self.inputs.append(input_ + [self.token2idx['<pad>']] * (max_len - len(input_)))
            else:
                self.inputs.append(input_[: max_len])

        self.labels = [[label] for label in df['label'].values.tolist()]
        self.n_class = len(df['label'].unique())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.LongTensor(self.inputs[idx]), torch.LongTensor(self.labels[idx])


class TextLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size, n_class, n_hidden):
        super().__init__()
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # rnn层，深度为1
        self.lstm = nn.LSTM(embed_size, n_hidden, 1, batch_first=True)
        # 激活函数
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        # 输出头
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, x):  # x: [batch_size * 句子长度]
        x = self.embedding(x)  # [batch_size * 句子长度 * embed_size]
        out, (_, _) = self.lstm(x)  # [batch_size * 句子长度 * n_hidden]
        out = self.relu(out)
        out = self.dropout(out)
        logits = self.fc(out[:,-1,:])  # 全连接输出头，[batch_size * n_class]
        return logits

In [3]:
dataset = MyDataset(max_len=10)  # 构造长度为10的句子输入，超过10的句子切掉，不足10的补<pad>
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
model = TextLSTM(vocab_size=dataset.vocab_size, embed_size=128,
                 n_class=dataset.n_class, n_hidden=256)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
for epoch in range(50):
    for feature, target in dataloader:
        optimizer.zero_grad()
        logits = model(feature)
        loss = criterion(logits, target.squeeze())
        loss.backward()
        optimizer.step()
    print('epoch:', epoch + 1, ', loss:', loss.item())

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/d1/4_gsqv2176z583_7rmpm27lh0000gn/T/jieba.cache
Loading model cost 0.403 seconds.
Prefix dict has been built successfully.


TextLSTM(
  (embedding): Embedding(8294, 128)
  (lstm): LSTM(128, 256, batch_first=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=256, out_features=14, bias=True)
)
epoch: 1 , loss: 2.243722915649414
epoch: 2 , loss: 1.5689152479171753
epoch: 3 , loss: 2.1620559692382812
epoch: 4 , loss: 1.585006594657898
epoch: 5 , loss: 0.7580963969230652
epoch: 6 , loss: 0.8219630122184753
epoch: 7 , loss: 0.3056076467037201
epoch: 8 , loss: 0.1576070338487625
epoch: 9 , loss: 0.0671982690691948
epoch: 10 , loss: 0.02522658184170723
epoch: 11 , loss: 0.006851658690720797
epoch: 12 , loss: 0.002321559004485607
epoch: 13 , loss: 0.0036406663712114096
epoch: 14 , loss: 0.0019566321279853582
epoch: 15 , loss: 0.003775387303903699
epoch: 16 , loss: 0.0025894646532833576
epoch: 17 , loss: 0.001988558564335108
epoch: 18 , loss: 0.002796419430524111
epoch: 19 , loss: 0.001974956365302205
epoch: 20 , loss: 0.0021424838341772556
epoch: 21 , loss: 0.000746710516978

In [4]:
df = pd.read_csv('../../datasets/THUCNews/train.csv')
predict = model(feature).max(1)[1].tolist()
for i in range(len(feature.tolist())):
    print(' '.join([dataset.idx2token[idx] for idx in feature.tolist()[i]]),
          '---> label:',
          dict(zip(df['label'], df['class']))[predict[i]])

湘财 证券 做空 能量 主导 大盘走势 <pad> <pad> <pad> <pad> ---> label: 股票
中联重科 全球 发售 索罗斯 认购 <pad> <pad> <pad> <pad> <pad> ---> label: 股票
酷酷 影更 精彩 寸 创维 液晶 仅 <pad> <pad> <pad> ---> label: 科技
星展 唯高达 六福 集团 首 评为 买进 <pad> <pad> <pad> ---> label: 股票
聚会 合影 选择 市售 超广角 相机 推荐 <pad> <pad> <pad> ---> label: 科技
图文 安富利 精英赛 第三轮 汤姆森 得力助手 <pad> <pad> <pad> <pad> ---> label: 体育
可能 会 惊喜 夏普 智能手机 年底 进 中国 <pad> <pad> ---> label: 科技
日本 经济 产业 相因 称 核电站 周边 死城 递交 辞呈 ---> label: 时政
欧 央行 月 日 停止 瑞郎 流动性 操作 <pad> <pad> ---> label: 股票
天津 高 考生 锐减 预计 录取率 增个 百分点 <pad> <pad> ---> label: 教育
招行 收购 永隆 商誉 减值 已达 永隆 扭亏 <pad> <pad> ---> label: 股票
瑞典 潜水 队 发现 藏身 海底 年 香槟 <pad> <pad> ---> label: 时政
世博 歌曲 曲 作者 回应 抄袭 质疑 <pad> <pad> <pad> ---> label: 娱乐
发改委 官员 称 正在 研究 出台 应对 经济 下行 措施 ---> label: 时政
中国 国航 国际 客货 供给 增速 快 需求 <pad> <pad> ---> label: 股票
阿联 那次 封盖 没 犯规 博古 特 特点 鲜明 <pad> ---> label: 体育
